Weight

In [1]:
import numpy as np

w = np.loadtxt("w0.fc1.weight.csv", delimiter=",")

w = w.reshape(784,128)

In [2]:
w.shape

(784, 128)

In [7]:
w[:16][:8]

array([[  1.,   2.,  -1., ..., -14.,   1.,  -5.],
       [ -1.,  14.,   0., ...,  10., -10.,   2.],
       [  5.,   0.,  12., ...,   5., -19., -15.],
       ...,
       [ -2.,  10.,   5., ...,   1.,  11.,   7.],
       [ 11.,  -6.,  -6., ..., -11.,  -3.,   0.],
       [  5.,   8.,  -2., ...,  10.,  -5.,   1.]], shape=(8, 128))

In [28]:
def make_instr(opvalid, opcode, sel, addr, data):
    return ((opvalid & 1) << 31) | ((opcode & 7) << 28) | \
           ((sel & 0xF) << 24) | ((addr & 0xFFFF) << 8) | (data & 0xFF)

In [ ]:
OPCODE_LDSRAM = 0x2
NUM_BANK = 4
OC = 128
IC = 784

instr = []

for oc in range(OC):
    bank = oc % NUM_BANK          # 0~3 반복
    bank_row = oc // NUM_BANK     # OC=0,4,8... → row 증가

    for ic in range(IC):
        addr = bank_row * IC + ic   # 각 row는 784 길이
        d = int(w[ic][oc]) & 0xFF
        inst = make_instr(1, OPCODE_LDSRAM, bank, addr, d)
        instr.append(inst)

In [30]:
with open("instr_mem_temp.hex", "w") as f:
    for inst in instr:
        f.write(f"{inst:08x}\n")

MNIST

In [36]:
import numpy as np

# CSV: label + 784 pixels
data = np.loadtxt("mnist_test_small copy.csv", delimiter=",")
labels = data[:, 0].astype(int)
inputs = data[:, 1:17]           # shape = (10, 784) for 10 images

In [38]:
inputs.shape

(10, 16)

In [40]:
def make_instr(opvalid, opcode, sel, addr, data):
    return ((opvalid & 1) << 31) | ((opcode & 7) << 28) | \
           ((sel & 0xF) << 24) | ((addr & 0xFFFF) << 8) | (data & 0xFF)

# 첫 번째 MNIST 이미지 사용 (784개)
img = inputs[0]             # shape = (784,)
img_q = img.astype(np.int8) # 또는 네 quant 함수 적용

NUM_BANK = 8
input_instr = []

for idx in range(16):
    bank = idx % NUM_BANK
    addr = idx // NUM_BANK   # integer division
    data = int(img_q[idx]) & 0xFF  # 반드시 0~255
    inst = make_instr(1, OPCODE_LDSRAM, bank, addr, data)
    input_instr.append(inst)

In [41]:
with open("instr_input_temp.hex", "w") as f:
    for inst in input_instr:
        f.write(f"{inst:08x}\n")

Result

In [61]:
import numpy as np

w = np.loadtxt("w0.fc1.weight.csv", delimiter=",")

w = w[:16*8]
w.shape
w = w.reshape(16,8)


# CSV: label + 784 pixels
data = np.loadtxt("mnist_test_small copy.csv", delimiter=",")
labels = data[:, 0].astype(int)
inputs = data[:, 1:17]           # shape = (10, 784) for 10 images

img = inputs[0]

print(w.shape)
print(img.shape)

(16, 8)
(16,)


In [62]:
img_row = img.reshape(1, 16)
out = img_row @ w    # (1,128)

In [63]:
img_row

array([[1., 2., 3., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [64]:
w

array([[  1.,   2.,  -1.,   1.,   1.,  -4.,   0., -10.],
       [  4.,   4.,  -6.,  11.,  11.,   2.,   0.,   4.],
       [  5.,   7.,  -8.,  -9.,  -5.,  11.,  -7.,  11.],
       [  9.,  11.,  -5.,   8.,   4.,   1., -10.,  11.],
       [  3.,  -6.,  -7.,  12.,   6.,  -9., -12.,   0.],
       [ -2.,   4.,  -4., -12., -10.,   1.,  -7.,   0.],
       [  5.,  10.,  -9., -11., -12.,  -6.,  -5.,   3.],
       [ -8.,  -6.,  -2.,   4.,  -9.,  -6.,  -8.,  -1.],
       [ 10.,   1.,  -9.,   1., -11.,  -9.,   6.,   5.],
       [  6.,   7.,   5.,  12., -12.,   8.,  -2.,  -1.],
       [  4.,   5.,  -4.,   6., -12.,   1.,   4., -12.],
       [  2.,  10.,  -3.,  -2.,  -1.,  11.,  12.,   8.],
       [-12., -11.,  -4.,   2.,   6., -10.,   5.,   5.],
       [ -6.,  -8.,  -9.,  12.,   4., -12.,  12.,   0.],
       [  3.,  12.,   3.,   2., -12.,  12.,   7., -11.],
       [  9.,   5.,  14., -10.,   5., -14.,   1.,  -5.]])

In [65]:
print(out)
print(out.shape)

[[ 24.  31. -37.  -4.   8.  33. -21.  31.]]
(1, 8)


1x8 하고 8x4만 때어서 계산

In [3]:
# ------------------------------
# 1) 입력 앞부분 8개 추출 → shape (8,)
# ------------------------------
img_part = img[:8]

# ------------------------------
# 2) weight 앞부분 8×4 추출 → shape (8,4)
# ------------------------------
w_part = w[:8, :4]

# ------------------------------
# 3) 행렬곱 (1×8) · (8×4) → (1×4)
# ------------------------------
out = img_part @ w_part   # 또는 np.dot(img_part, w_part)

print("img_part shape =", img_part.shape)   # (8,)
print("w_part shape =", w_part.shape)       # (8, 4)
print("out shape =", out.shape)             # (4,)
print(out)

img_part shape = (8,)
w_part shape = (8, 4)
out shape = (4,)
[0. 0. 0. 0.]


In [7]:
# ------------------------------
# 1) 전체 입력 1×784 → shape (784,)
# ------------------------------
img_full = img   # 이미 img = inputs[0] 이므로 (784,)

# ------------------------------
# 2) weight 앞부분 784×4 추출 → shape (784,4)
# ------------------------------
w_part = w[:, :4]

# ------------------------------
# 3) 행렬곱 (1×784) · (784×4) → (1×4)
# ------------------------------
out = img_full @ w_part   # 또는 np.dot(img_full, w_part)

print("img shape =", img_full.shape)   # (784,)
print("w_part shape =", w_part.shape)  # (784, 4)
print("out shape =", out.shape)        # (4,)
print(out)

img shape = (784,)
w_part shape = (784, 4)
out shape = (4,)
[19043.  3477. 36001. 23166.]
